In [1]:
# Import packages
import geopandas as gpd
import numpy as np
import pandas as pd
import libpysal
import networkx as nx
import osmnx as ox
import time
from shapely import geometry
from shapely.geometry import Point, MultiLineString, LineString, Polygon
from shapely.ops import nearest_points
import matplotlib.pyplot as plt
from itertools import product, combinations
import multiprocessing as mp
import math
import warnings
# Import some libraries
import socket
from wpgpDownload.utils.dl import wpFtp
from wpgpDownload.utils.isos import Countries
from wpgpDownload.utils.convenience_functions import download_country_covariates as dl
from wpgpDownload.utils.wpcsv import Product
import sys
import georasters as gr
import rasterio
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
# Block 0 cities and assumptions

start = time.time()

cities = ['Philadelphia, United States','Denver, United States','Ghent, Belgium','Amsterdam, Netherlands',
         'Dhaka Metropolitan, Bangladesh','Dublin, Ireland','Vancouver, Canada',
          'Tel Aviv, Israel','Washington DC, United States']

#cities = ['Greater London, United Kingdom','Tel Aviv, Israel']

# Assumptions
thresholds = [300, 600, 1000] # route threshold in metres. WHO guideline speaks of access within 300m
walkable_park_dist = 500 # radius in metres
one_park_buffer = 25 # in metres
park_entry_point_buffer = 25 # in metres
min_park_size = 400 # in squared metres (WHO = 0.04 ha = 400m)
park_entry_point_merge = 0 # (default)

thresholds
# 'Dublin, Ireland' is problematic; road system is too dense, has separate script

[300, 600, 1000]

In [3]:
# Block 1 Extract city boundaries, download grids in cities' country

warnings.filterwarnings("ignore")
bound_df = pd.DataFrame()
for i in cities:
    bound_df = pd.concat([bound_df, pd.DataFrame(ox.geocoder.geocode_to_gdf(i))])
    
bound_df = bound_df.reset_index()
bound_df = bound_df.loc[:,bound_df.columns!='index']
bound_df = gpd.GeoDataFrame(bound_df, geometry = 'geometry', crs = 4326)
bound_df.to_file(r'D:/Dumps/City_boundaries/Boundaries_osm.shp')

start_time = time.time()

# Get unique ISO countries, so all country-grids are only loaded once (i.e. Philadelphia, Denver and Washington DC in the USA)
C = pd.DataFrame(Countries)
start_time = time.time()
iso_countries = []
for i in range(len(cities)): 
    countries = bound_df['display_name'][i].rsplit(',')[-1][1:]
    iso = C[C['name'] == countries].iloc[0,1]
    if iso not in iso_countries:
        iso_countries.append(iso)

blocks = []
for i in iso_countries:
    path = 'D:\\Dumps\\WorldPoP_Grids\\' + i.lower() + '_ppp_2020.tif'
    block = gr.from_file(path)
    blocks.append(block)
    print(i,'extracted', round((time.time() - start_time)/60,2),'mns')
bound_df

USA extracted 1.18 mns
BEL extracted 1.18 mns
NLD extracted 1.19 mns
BGD extracted 1.26 mns
IRL extracted 1.33 mns
CAN extracted 4.58 mns
ISR extracted 4.6 mns


,geometry,bbox_north,bbox_south,bbox_east,bbox_west,place_id,osm_type,osm_id,lat,lon,display_name,class,type,importance
0,"POLYGON ((-75.28030 39.97500, -75.28022 39.974...",40.137959,39.867005,-74.955831,-75.280298,282310523,relation,188022,40.002410,-75.139364,"Philadelphia, Philadelphia County, Pennsylvani...",boundary,administrative,1.023797
1,"POLYGON ((-105.10988 39.62710, -105.10761 39.6...",39.914209,39.614315,-104.599700,-105.109885,327393456,relation,1411339,39.739236,-104.984862,"Denver, Colorado, United States",boundary,administrative,0.905476
2,"POLYGON ((3.57976 51.03027, 3.57992 51.03022, ...",51.188886,50.979542,3.849325,3.579762,282032351,relation,897671,51.084199,3.732851,"Ghent, Gent, East Flanders, Flanders, Belgium",boundary,administrative,0.855558
3,"MULTIPOLYGON (((4.72876 52.40071, 4.73371 52.4...",52.431064,52.278174,5.079162,4.728756,282057196,relation,271110,52.372760,4.893604,"Amsterdam, North Holland, Netherlands",boundary,administrative,0.946813
4,"POLYGON ((90.32978 23.75081, 90.32981 23.75050...",23.899531,23.668077,90.508832,90.329784,328693247,relation,13663697,23.783663,90.403246,"Dhaka Metropolitan, Dhaka District, Dhaka Divi...",boundary,administrative,0.710000
5,"POLYGON ((-6.38703 53.34081, -6.38658 53.33867...",53.410542,53.298734,-6.114883,-6.387026,282061541,relation,1109531,53.349764,-6.260273,"Dublin, Dublin 1, Leinster, Ireland",boundary,administrative,0.921590
6,"POLYGON ((-123.22496 49.27462, -123.22475 49.2...",49.316171,49.198445,-123.023242,-123.224961,282265567,relation,1852574,49.260872,-123.113952,"Vancouver, District of North Vancouver, Metro ...",boundary,administrative,0.906968
7,"POLYGON ((34.73913 32.03376, 34.74220 32.03297...",32.146977,32.029344,34.852262,34.739131,328372020,relation,1382494,32.085300,34.781806,"Tel Aviv-Yafo, Tel Aviv Subdistrict, Tel Aviv ...",boundary,administrative,0.959988
8,"POLYGON ((-77.11979 38.93435, -77.11977 38.934...",38.995968,38.791630,-76.909366,-77.119795,282620879,relation,5396194,38.895037,-77.036543,"Washington, District of Columbia, United States",boundary,administrative,1.059289


In [4]:
# Block 2 e
start_time = time.time()
clips = []
grids = []
print('300m resolution grids extraction')
for i in range(len(cities)):
    iso = C[cities[i].rsplit(',')[1][1:] == C['name']].iloc[0,1]
    contains = [j for j, x in enumerate(iso_countries) if x == iso][0]
    
    # Clip the city from the country
    clipped = blocks[contains].clip(bound_df['geometry'][i])
    clipped = clipped[0].to_geopandas()
    
    # Get dissolvement_key for 300m by 300m dissolvement. 
    clipped['row3'] = np.floor(clipped['row']/3).astype(int)
    clipped['col3'] = np.floor(clipped['col']/3).astype(int)
    clipped['dissolve_key'] = clipped['row3'].astype(str) +'-'+ clipped['col3'].astype(str)
    clips.append(clipped)
        
    # Dissolve into 300m by 300m grids
    popgrid = clipped[['dissolve_key','geometry','row3','col3']].dissolve('dissolve_key')

    # Get those grids populations and area. Only blocks with population and full blocks
    popgrid['population'] = round(clipped.groupby('dissolve_key')['value'].sum()).astype(int)
    popgrid['area_m'] = round(gpd.GeoSeries(popgrid['geometry'], crs = 4326).to_crs(3043).area).astype(int)
    popgrid = popgrid[popgrid['population'] > 0]
    popgrid = popgrid[popgrid['area_m'] / popgrid['area_m'].max() > 0.95]
    
    # Get centroids and coords
    popgrid['centroid'] = popgrid['geometry'].centroid
    popgrid['centroid_m'] = gpd.GeoSeries(popgrid['centroid'], crs = 4326).to_crs(3043)
    popgrid['grid_lon'] = popgrid['centroid_m'].x
    popgrid['grid_lat'] = popgrid['centroid_m'].y
    popgrid = popgrid.reset_index()
    
    minx = popgrid.bounds['minx']
    maxx = popgrid.bounds['maxx']
    miny = popgrid.bounds['miny']
    maxy = popgrid.bounds['maxy']

    # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters) which is in my mind an coords error
    # I therefore create one polygon
    Poly = []
    for k in range(len(popgrid)):
        Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
    popgrid['geometry'] = Poly
    
    grids.append(popgrid)
    
    print(cities[i].rsplit(',')[0], round((time.time() - start_time)/60,2),'mns')
    
grids[0]

300m resolution grids extraction
Philadelphia 0.21 mns
Denver 0.36 mns
Ghent 0.43 mns
Amsterdam 0.51 mns
Dhaka Metropolitan 0.58 mns
Dublin 0.64 mns
Vancouver 0.69 mns
Tel Aviv 0.7 mns
Washington DC 0.77 mns


,dissolve_key,geometry,row3,col3,population,area_m,centroid,centroid_m,grid_lon,grid_lat
0,1-105,"POLYGON ((-75.01792 40.13625, -75.01542 40.136...",1,105,134,133566,POINT (-75.01667 40.13500),POINT (-5675915.991 8459971.271),-5.675916e+06,8.459971e+06
1,1-106,"POLYGON ((-75.01542 40.13625, -75.01292 40.136...",1,106,160,133563,POINT (-75.01417 40.13500),POINT (-5675816.286 8459667.010),-5.675816e+06,8.459667e+06
2,1-107,"POLYGON ((-75.01292 40.13625, -75.01042 40.136...",1,107,77,133560,POINT (-75.01167 40.13500),POINT (-5675716.563 8459362.758),-5.675717e+06,8.459363e+06
3,10-100,"POLYGON ((-75.03042 40.11375, -75.02792 40.113...",10,100,449,133736,POINT (-75.02917 40.11250),POINT (-5679983.231 8460324.302),-5.679983e+06,8.460324e+06
4,10-101,"POLYGON ((-75.02792 40.11375, -75.02542 40.113...",10,101,682,133733,POINT (-75.02667 40.11250),POINT (-5679883.502 8460019.778),-5.679884e+06,8.460020e+06
...,...,...,...,...,...,...,...,...,...,...
4682,99-14,"POLYGON ((-75.24542 39.89125, -75.24292 39.891...",99,14,2,135549,POINT (-75.24417 39.89000),POINT (-5723954.143 8475126.598),-5.723954e+06,8.475127e+06
4683,99-15,"POLYGON ((-75.24292 39.89125, -75.24042 39.891...",99,15,1,135546,POINT (-75.24167 39.89000),POINT (-5723854.853 8474819.097),-5.723855e+06,8.474819e+06
4684,99-38,"POLYGON ((-75.18542 39.89125, -75.18292 39.891...",99,38,1,135474,POINT (-75.18417 39.89000),POINT (-5721566.085 8467749.306),-5.721566e+06,8.467749e+06
4685,99-39,"POLYGON ((-75.18292 39.89125, -75.18042 39.891...",99,39,1,135471,POINT (-75.18167 39.89000),POINT (-5721466.351 8467442.043),-5.721466e+06,8.467442e+06


In [5]:
# Block 3 Road networks

warnings.filterwarnings("ignore")

start_time = time.time()
graphs = list()
road_nodes = list()
road_edges = list()
road_conn = list()
#roads = list()

for i in cities:
    # Get graph, road nodes and edges
    graph = ox.graph_from_place(i, network_type="all", buffer_dist = (np.max(thresholds)+1000))
    graphs.append(graph)
    road_node, road_edge = ox.graph_to_gdfs(graph)
    
    # Road nodes format
    road_node = road_node.to_crs(4326)
    road_node['geometry_m'] = gpd.GeoSeries(road_node['geometry'], crs = 4326).to_crs(3043)
    road_node['osmid_var'] = road_node.index
    road_node = gpd.GeoDataFrame(road_node, geometry = 'geometry', crs = 4326)
    road_nodes.append(road_node)
    #road = road_node.reset_index()
    #roads.append(road)
    
    # format road edges
    road_edge = road_edge.to_crs(4326)
    road_edge['geometry_m'] = gpd.GeoSeries(road_edge['geometry'], crs = 4326).to_crs(3043)
    road_edge = road_edge.reset_index()
    road_edge.rename(columns={'u':'from', 'v':'to'}, inplace=True)
    road_edge['key'] = road_edge['from'].astype(str) + '-' + road_edge['to'].astype(str)
    road_edges.append(road_edge)
    
    # Get only necessary road connections columns for network performance
    road_con = road_edge[['osmid','key','length','geometry']]
    road_con = road_con.set_index('key')
    road_conn.append(road_con)
    print(i.rsplit(',')[0], 'done', round((time.time() - start_time) / 60,2),'mns')
road_edges[0]

Philadelphia done 1.95 mns
Denver done 5.04 mns
Ghent done 5.69 mns
Amsterdam done 7.2 mns
Dhaka Metropolitan done 7.97 mns
Dublin done 9.2 mns
Vancouver done 10.61 mns
Tel Aviv done 11.06 mns
Washington DC done 12.88 mns


,from,to,key,osmid,name,highway,oneway,length,geometry,ref,lanes,width,bridge,service,tunnel,maxspeed,access,junction,area,geometry_m
0,103237949,103353127,103237949-103353127,11591597,New Jersey Avenue,residential,False,111.799,"LINESTRING (-74.96444 40.03625, -74.96557 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689617..."
1,103237949,103353090,103237949-103353090,11591597,New Jersey Avenue,residential,False,107.489,"LINESTRING (-74.96444 40.03625, -74.96337 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689364..."
2,103237949,103237976,103237949-103237976,11580386,Cleveland Avenue,residential,False,168.717,"LINESTRING (-74.96444 40.03625, -74.96404 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689489.546 8448457.709, -5689552..."
3,103237976,103238007,103237976-103238007,11580386,Cleveland Avenue,residential,False,151.349,"LINESTRING (-74.96340 40.03496, -74.96245 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689797..."
4,103237976,103590312,103237976-103590312,11610261,2nd Street,residential,False,108.508,"LINESTRING (-74.96340 40.03496, -74.96448 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5689653.040 8448263.511, -5689779..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338097,9726316545,5550941352,9726316545-5550941352,579660432,NaN,service,False,39.965,"LINESTRING (-75.24128 40.06507, -75.24111 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695935..."
338098,9726316545,9726316548,9726316545-9726316548,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24128 40.06507, -75.24148 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5695926.465 8483769.053, -5695962..."
338099,9726316548,9726316545,9726316548-9726316545,1058487388,NaN,service,False,58.733,"LINESTRING (-75.24175 40.06470, -75.24175 40.0...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5696003.982 8483808.258, -5695998..."
338100,9728655691,110122187,9728655691-110122187,12149073,South 10th Street,residential,True,3.250,"LINESTRING (-75.15829 39.94437, -75.15830 39.9...",NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"LINESTRING (-5711864.888 8467388.116, -5711869..."


In [6]:
# Block 4 city greenspace
parks_in_range = list()
for i in cities:
    gdf = ox.geometries_from_place(i, tags={'leisure':'park'}, buffer_dist = np.max(thresholds))
    gdf = gdf[(gdf.geom_type == 'Polygon') | (gdf.geom_type == 'MultiPolygon')]
    greenspace = gdf.reset_index()
    #print('extracting done')
    #parks_osm.append(gdf)
    #concat = pd.concat([gdf['osmid'],gdf['name'],gdf['element_type'],gdf['geometry']], axis = 1)
    #concat = gpd.GeoDataFrame(concat, geometry = 'geometry', crs = 4326)
    #concat.to_file('D:/Dumps/Greenspace_OSM/parks_'+i.rsplit(',')[0]+'.gpkg')
    warnings.filterwarnings("ignore")
    
    green_buffer = gpd.GeoDataFrame(geometry = greenspace.to_crs(3043).buffer(one_park_buffer).to_crs(4326))
    greenspace['geometry_w_buffer'] = green_buffer
    greenspace['geometry_w_buffer'] = gpd.GeoSeries(greenspace['geometry_w_buffer'], crs = 4326)
    greenspace['geom buffer diff'] = greenspace['geometry_w_buffer'].difference(greenspace['geometry'])

    # This function group components in itself that overlap (with the buffer set of 25 metres)
    # https://stackoverflow.com/questions/68036051/geopandas-self-intersection-grouping
    W = libpysal.weights.fuzzy_contiguity(greenspace['geometry_w_buffer'])
    greenspace['components'] = W.component_labels
    parks = greenspace.dissolve('components')
    
    # Exclude parks below 0.04 ha.
    parks = parks[parks.to_crs(3043).area > min_park_size]
    print(i, 'done')
    parks = parks.reset_index()
    parks_in_range.append(parks)

extracting done
Philadelphia, United States done
extracting done
Denver, United States done
extracting done
Ghent, Belgium done
extracting done
Amsterdam, Netherlands done
extracting done
Dhaka Metropolitan, Bangladesh done
extracting done
Dublin, Ireland done
extracting done
Vancouver, Canada done
extracting done
Tel Aviv, Israel done
extracting done
Washington DC, United States done


In [7]:
# Block 5 park entry points
start_time = time.time()
ParkRoads = list()
for j in range(len(cities)):
    ParkRoad = pd.DataFrame()
    mat = list()
    # For all
    for i in range(len(parks_in_range[j])):
        dist = road_nodes[j]['geometry'].to_crs(3043).distance(parks_in_range[j]['geometry'].to_crs(
            3043)[i])
        buf_nodes = road_nodes[j][(dist < park_entry_point_buffer) & (dist > 0)]
        mat.append(list(np.repeat(i, len(buf_nodes))))
        ParkRoad = pd.concat([ParkRoad, buf_nodes])
        if i % 100 == 0: print(cities[j].rsplit(',')[0], round(i/len(parks_in_range[j])*100,1),'% done', 
                              round((time.time() - start_time) / 60,2),' mns')
    # Park no list conversion
    mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, mat) for i in b]

    # Format
    ParkRoad['Park_No'] = mat_u
    ParkRoad = ParkRoad.reset_index()
    ParkRoad['park_lon'] = ParkRoad['geometry_m'].x
    ParkRoad['park_lat'] = ParkRoad['geometry_m'].y
    
    # Get the road nodes intersecting with the parks' buffer
    ParkRoad = pd.merge(ParkRoad, parks_in_range[j][['geometry']], left_on = 'Park_No', right_index = True)
    #ParkRoad = pd.merge(ParkRoad, parks_in_range[j][thresholds_str], left_on = 'Park_No', right_index = True)
    
    # Get the walkable park size
    ParkRoad['park_size_walkable'] = ParkRoad['geometry_m'].buffer(walkable_park_dist).to_crs(4326).intersection(ParkRoad['geometry_y'])
    ParkRoad['walk_area'] = ParkRoad['park_size_walkable'].to_crs(3043).area
    ParkRoad['park_area'] = ParkRoad['geometry_y'].to_crs(3043).area
    ParkRoad['share_walked'] = ParkRoad['walk_area'] / ParkRoad['park_area']
    
    # Get size inflation factors for the gravity model
    ParkRoad['size_infl_factor'] = ParkRoad['walk_area'] / ParkRoad['walk_area'].median()
    ParkRoad['size_infl_proot2'] = ParkRoad['size_infl_factor']**(1/2)
    ParkRoad['size_infl_proot3'] = ParkRoad['size_infl_factor']**(1/3)
    ParkRoad['size_infl_proot5'] = ParkRoad['size_infl_factor']**(1/5)
    ParkRoads.append(ParkRoad)
    
    fltr = ParkRoad.loc[:,~ParkRoad.columns.isin(['geometry_x','geometry_m','park_size_walkable', 
                                                          'geometry_m_buffer'])]
                     
    gdf = gpd.GeoDataFrame(pd.DataFrame(fltr), geometry = 'geometry_y', crs = 4326)
    gdf.to_file('D:Dumps/Scores output OSM/Park_entry_points/'+ cities[j] +'.shp')
    
    print(cities[j].rsplit(',')[0],'100 % done', 
                              round((time.time() - start_time) / 60,2),' mns')
    
ParkRoads[0]

Philadelphia 0.0 % done 0.02  mns
Philadelphia 22.3 % done 0.65  mns
Philadelphia 44.5 % done 1.24  mns
Philadelphia 66.8 % done 1.77  mns
Philadelphia 89.1 % done 2.26  mns
Philadelphia 100 % done 2.64  mns
Denver 0.0 % done 2.66  mns
Denver 29.4 % done 3.79  mns
Denver 58.8 % done 4.65  mns
Denver 88.2 % done 5.49  mns
Denver 100 % done 5.97  mns
Ghent 0.0 % done 5.97  mns
Ghent 42.6 % done 6.19  mns
Ghent 85.1 % done 6.38  mns
Ghent 100 % done 6.48  mns
Amsterdam 0.0 % done 6.48  mns
Amsterdam 57.1 % done 6.98  mns
Amsterdam 100 % done 7.47  mns
Dhaka Metropolitan 0.0 % done 7.47  mns
Dhaka Metropolitan 59.5 % done 7.68  mns
Dhaka Metropolitan 100 % done 7.82  mns
Dublin 0.0 % done 7.83  mns
Dublin 33.1 % done 8.26  mns
Dublin 66.2 % done 8.64  mns
Dublin 99.3 % done 8.99  mns
Dublin 100 % done 9.03  mns
Vancouver 0.0 % done 9.04  mns
Vancouver 42.0 % done 9.51  mns
Vancouver 84.0 % done 9.93  mns
Vancouver 100 % done 10.14  mns
Tel Aviv 0.0 % done 10.14  mns
Tel Aviv 25.3 % done 10

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_lat,geometry_y,park_size_walkable,walk_area,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5
0,109729372,39.956835,-75.147299,4,NaN,NaN,POINT (-75.14730 39.95683),POINT (-5709440.723 8466685.277),109729372,0,...,8.466685e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14345 39.95648, -75.14348 ...",129260.205233,766205.364911,0.168702,1.724732,1.313291,1.199243,1.115178
1,109729474,39.952591,-75.148474,4,traffic_signals,NaN,POINT (-75.14847 39.95259),POINT (-5710163.665 8466609.499),109729474,0,...,8.466609e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14638 39.95008, -75.14671 ...",152813.334108,766205.364911,0.199442,2.039004,1.427937,1.268059,1.153144
2,109729486,39.953330,-75.148318,5,NaN,NaN,POINT (-75.14832 39.95333),POINT (-5710039.650 8466628.565),109729486,0,...,8.466629e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14539 39.95528, -75.14543 ...",173200.754874,766205.364911,0.226050,2.311035,1.520209,1.322114,1.182392
3,109729590,39.955551,-75.147899,3,NaN,NaN,POINT (-75.14790 39.95555),POINT (-5709669.114 8466692.323),109729590,0,...,8.466692e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14429 39.95664, -75.14419 ...",176718.973190,766205.364911,0.230642,2.357979,1.535571,1.331006,1.187157
4,109729661,39.956254,-75.147625,4,NaN,NaN,POINT (-75.14763 39.95625),POINT (-5709546.213 8466695.127),109729661,0,...,8.466695e+06,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...","MULTIPOLYGON (((-75.14376 39.95649, -75.14375 ...",160141.739542,766205.364911,0.209006,2.136787,1.461775,1.288014,1.163998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,7098423781,39.966995,-75.128847,1,NaN,NaN,POINT (-75.12885 39.96700),POINT (-5707086.662 8464950.373),7098423781,448,...,8.464950e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5660,7098423782,39.966951,-75.128758,4,crossing,NaN,POINT (-75.12876 39.96695),POINT (-5707090.110 8464937.109),7098423782,448,...,8.464937e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5661,8151346056,39.967180,-75.128285,3,NaN,NaN,POINT (-75.12829 39.96718),POINT (-5707034.735 8464891.123),8151346056,448,...,8.464891e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929
5662,8912442744,39.967246,-75.128846,4,NaN,NaN,POINT (-75.12885 39.96725),POINT (-5707046.718 8464963.244),8912442744,448,...,8.464963e+06,"MULTIPOLYGON (((-75.12728 39.96599, -75.12739 ...","MULTIPOLYGON (((-75.12739 39.96589, -75.12747 ...",73431.800154,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929


In [8]:
# Block 5.5 (not in use, buffer is 0, thus retains all the park entry points as is)

# Get buffer of nodes close to each other.
ParkCombs = list([])
for i in range(len(cities)):
    
    # Get the buffer
    ParkComb = ParkRoads[i]
    ParkComb['geometry_m_buffer'] = ParkComb['geometry_m'].buffer(park_entry_point_merge)
    
    # Get and merge components
    M = libpysal.weights.fuzzy_contiguity(ParkComb['geometry_m_buffer'])
    ParkComb['components'] = M.component_labels
    
    # Take centroid of merged components
    centr = gpd.GeoDataFrame(ParkComb, geometry = 'geometry_x', crs = 4326).dissolve('components')['geometry_x'].centroid
    centr = gpd.GeoDataFrame(centr)
    centr.columns = ['comp_centroid']
    
    # Get node closest to the centroid of all merged nodes, which accesses the road network.
    ParkComb = pd.merge(ParkComb, centr, left_on = 'components', right_index = True)
    ParkComb['centr_dist'] = ParkComb['geometry_x'].distance(ParkComb['comp_centroid'])
    ParkComb = ParkComb.iloc[ParkComb.groupby('components')['centr_dist'].idxmin()]
    ParkCombs.append(ParkComb)
ParkCombs[0]

,osmid,y,x,street_count,highway,ref,geometry_x,geometry_m,osmid_var,Park_No,...,park_area,share_walked,size_infl_factor,size_infl_proot2,size_infl_proot3,size_infl_proot5,geometry_m_buffer,components,comp_centroid,centr_dist
0,109729372,39.956835,-75.147299,4,NaN,NaN,POINT (-75.14730 39.95683),POINT (-5709440.723 8466685.277),109729372,0,...,766205.364911,0.168702,1.724732,1.313291,1.199243,1.115178,POLYGON EMPTY,0,POINT (-75.14730 39.95683),0.0
1,109729474,39.952591,-75.148474,4,traffic_signals,NaN,POINT (-75.14847 39.95259),POINT (-5710163.665 8466609.499),109729474,0,...,766205.364911,0.199442,2.039004,1.427937,1.268059,1.153144,POLYGON EMPTY,1,POINT (-75.14847 39.95259),0.0
2,109729486,39.953330,-75.148318,5,NaN,NaN,POINT (-75.14832 39.95333),POINT (-5710039.650 8466628.565),109729486,0,...,766205.364911,0.226050,2.311035,1.520209,1.322114,1.182392,POLYGON EMPTY,2,POINT (-75.14832 39.95333),0.0
3,109729590,39.955551,-75.147899,3,NaN,NaN,POINT (-75.14790 39.95555),POINT (-5709669.114 8466692.323),109729590,0,...,766205.364911,0.230642,2.357979,1.535571,1.331006,1.187157,POLYGON EMPTY,3,POINT (-75.14790 39.95555),0.0
4,109729661,39.956254,-75.147625,4,NaN,NaN,POINT (-75.14763 39.95625),POINT (-5709546.213 8466695.127),109729661,0,...,766205.364911,0.209006,2.136787,1.461775,1.288014,1.163998,POLYGON EMPTY,4,POINT (-75.14763 39.95625),0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5659,7098423781,39.966995,-75.128847,1,NaN,NaN,POINT (-75.12885 39.96700),POINT (-5707086.662 8464950.373),7098423781,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5659,POINT (-75.12885 39.96700),0.0
5660,7098423782,39.966951,-75.128758,4,crossing,NaN,POINT (-75.12876 39.96695),POINT (-5707090.110 8464937.109),7098423782,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5660,POINT (-75.12876 39.96695),0.0
5661,8151346056,39.967180,-75.128285,3,NaN,NaN,POINT (-75.12829 39.96718),POINT (-5707034.735 8464891.123),8151346056,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5661,POINT (-75.12829 39.96718),0.0
5662,8912442744,39.967246,-75.128846,4,NaN,NaN,POINT (-75.12885 39.96725),POINT (-5707046.718 8464963.244),8912442744,448,...,73431.800154,1.000000,0.979808,0.989852,0.993223,0.995929,POLYGON EMPTY,5662,POINT (-75.12885 39.96725),0.0


In [9]:
# Block 6 grid-parkentry combinations within euclidean threshold distance

start_time = time.time()
RoadComb = list()
for l in range(len(cities)):
    block = 1000
    print(cities[l])
    # Check all parks within blockm radius
    len1 = len(grids[l])
    len2 = len(ParkCombs[l])
    len3 = int(np.ceil(len2/block))
    output = pd.DataFrame()
    len_mat = 0
    # Checking all the combinations at once is too performance intensive, it is broken down per 1000 (or what you want)
    for i in range(len3):
        # Check all grid-park combinations per block
        l1, l2 = range(0,len1), range(i*block,(i+1)*block)
        listed = pd.DataFrame(list(product(l1, l2)))

        # Merge grid and park information
        grid_merged = pd.merge(listed, 
                               grids[l][['grid_lon','grid_lat','centroid','centroid_m']],
                               left_on = 0, right_index = True)
        node_merged = pd.merge(grid_merged, 
                               ParkCombs[l][['Park_No','osmid','geometry_x','geometry_y','geometry_m','park_lon','park_lat',
                                   'size_infl_proot2','size_infl_proot3','size_infl_proot5','share_walked','park_area','walk_area']], 
                               left_on = 1, right_index = True)

        # Preset index for merging
        node_merged['key'] = range(0,len(node_merged))
        node_merged = node_merged.set_index('key')
        node_merged = node_merged.loc[:, ~node_merged.columns.isin(['index'])]

        # Create lists for better computational performance
        glon = list(node_merged['grid_lon'])
        glat = list(node_merged['grid_lat'])
        plon = list(node_merged['park_lon'])
        plat = list(node_merged['park_lat'])
        infl2 = list(node_merged['size_infl_proot2'])
        infl3 = list(node_merged['size_infl_proot3'])
        infl5 = list(node_merged['size_infl_proot5'])

        # Get the euclidean distances
        mat = list()
        mat2 = list()
        mat3 = list()
        mat4 = list()
        for j in range(len(node_merged)):
            mat.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2))
            mat2.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl2[j])
            mat3.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl3[j])
            mat4.append(math.sqrt(abs(plon[j] - glon[j])**2 + abs(plat[j] - glat[j])**2) / infl5[j])

        # Check if distances are within 1000m and join remaining info and concat in master df per 1000.
        mat_df = pd.DataFrame(mat3)[(np.array(mat) <= np.max(thresholds)) | 
                                    (np.array(mat2) <= np.max(thresholds)) | 
                                    (np.array(mat3) <= np.max(thresholds)) | 
                                    (np.array(mat4) <= np.max(thresholds))]

        # join the other gravity euclidean scores and other information
        mat_df = mat_df.join(pd.DataFrame(mat), lsuffix='_infl', rsuffix='_entr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat2), lsuffix='_entry', rsuffix='_pwr', how = 'left')
        mat_df = mat_df.join(pd.DataFrame(mat4), lsuffix='_pwr', rsuffix='_root', how = 'left')
        mat_df.columns = ['size_infl_eucl2','raw euclidean','size_infl_eucl3','size_infl_eucl5']    
        mat_df = mat_df.join(node_merged)

        output = pd.concat([output, mat_df])

        if ((i+1) % 1 == 0) | ((i+1) == len3): print((i+1),'/',len3,'comb. done',round((time.time() - start_time) / 60,2),' mns')
        if ((i+1) % 1 == 0) | ((i+1) == len3): print('of',np.where(i+1 == len3, len2 % block * block, len1*block) ,
                                                     'within a Gravity model variant in one of',thresholds,'m threshold:',len(mat_df))

        # Checks the number of the parks within 1000m.
        len_mat = len_mat + len(mat_df)

    # Renaming columns
    print('total combinations within distance',len_mat)
    
    output.columns = ['size_infl_eucl3','raw euclidean','size_infl_eucl2','size_infl_eucl5',
                      'Grid_No','Park_entry_No','grid_lon','grid_lat','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                      'park_lon','park_lat','size_infl_proot2','size_infl_proot3','size_infl_proot5',
                      'parkshare_walked','park_area','walk_area_m2']
    output = output[['raw euclidean','size_infl_eucl2','size_infl_eucl3','size_infl_eucl5',
                     'Grid_No','Park_entry_No','Grid_coords_centroid','Grid_m_centroid',
                      'Park_No','Parkroad_osmid','Park_geom','Parkroad_coords_centroid','Parkroad_m_centroid',
                     'walk_area_m2','size_infl_proot2','size_infl_proot3','size_infl_proot5']]
    
    # Reinstate geographic elements
    output = gpd.GeoDataFrame(output, geometry = 'Grid_coords_centroid', crs = 4326)
    output['Grid_m_centroid'] = gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043)
    output['Parkroad_coords_centroid'] = gpd.GeoSeries(output['Parkroad_coords_centroid'], crs = 4326)
    output['Parkroad_m_centroid'] = gpd.GeoSeries(output['Parkroad_m_centroid'], crs = 3043)
    
    # Get the nearest entrance point for the grid centroids
    mat5 = list()
    for i in range(len(output)):
        nearest = int(road_nodes[l]['geometry'].sindex.nearest(output['Grid_coords_centroid'].iloc[i])[1])
        mat5.append(road_nodes[l]['osmid_var'].iloc[nearest])
        if i % 250000 == 0: print(round(i/len(output)*100,1),'% gridentry done', round((time.time() - start_time) / 60,2),' mns')
            
    # format resulting dataframe
    output['grid_osm'] = mat5
    output = pd.merge(output, road_nodes[l]['geometry'], left_on = 'grid_osm', right_index = True)
    output['geometry_m'] = gpd.GeoSeries(output['geometry'], crs = 4326).to_crs(3043)
    output['grid_entry_dist'] = round(gpd.GeoSeries(output['Grid_m_centroid'], crs = 3043
                                                   ).distance(output['geometry_m']),3)
    output = output.reset_index()
    print('100 % gridentry done', round((time.time() - start_time) / 60,2),' mns')
    RoadComb.append(output)
RoadComb[0]


Philadelphia, United States
1 / 6 comb. done 0.46  mns
of 4687000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 48887
2 / 6 comb. done 0.95  mns
of 4687000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 47354
3 / 6 comb. done 1.43  mns
of 4687000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 20509
4 / 6 comb. done 1.9  mns
of 4687000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 27146
5 / 6 comb. done 2.37  mns
of 4687000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 21904
6 / 6 comb. done 2.71  mns
of 664000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 22103
total combinations within distance 187903
0.0 % gridentry done 2.72  mns
100 % gridentry done 3.01  mns
Denver, United States
1 / 7 comb. done 3.41  mns
of 3857000 within a Gravity model variant in one of [300, 600, 1000] m threshold: 49271
2 / 7 comb. done 3.81  mns
of 3857000 withi

,index,raw euclidean,size_infl_eucl2,size_infl_eucl3,size_infl_eucl5,Grid_No,Park_entry_No,Grid_coords_centroid,Grid_m_centroid,Park_No,...,Parkroad_coords_centroid,Parkroad_m_centroid,walk_area_m2,size_infl_proot2,size_infl_proot3,size_infl_proot5,grid_osm,geometry,geometry_m,grid_entry_dist
0,3838,1297.698919,988.127838,1082.098356,1163.669415,3838,0,POINT (-75.15417 39.96250),POINT (-5708811.866 8467820.426),0,...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709440.723 8466685.277),129260.205233,1.313291,1.199243,1.115178,5559104498,POINT (-75.15400 39.96258),POINT (-5708792.439 8467804.519),25.108
1,17899,1416.858745,922.691622,1064.502216,1193.489394,3838,3,POINT (-75.15417 39.96250),POINT (-5708811.866 8467820.426),0,...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709669.114 8466692.323),176718.973190,1.535571,1.331006,1.187157,5559104498,POINT (-75.15400 39.96258),POINT (-5708792.439 8467804.519),25.108
2,22586,1343.711403,919.232444,1043.243203,1154.393145,3838,4,POINT (-75.15417 39.96250),POINT (-5708811.866 8467820.426),0,...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709546.213 8466695.127),160141.739542,1.461775,1.288014,1.163998,5559104498,POINT (-75.15400 39.96258),POINT (-5708792.439 8467804.519),25.108
3,27273,1278.582788,951.389858,1049.901677,1136.006246,3838,5,POINT (-75.15417 39.96250),POINT (-5708811.866 8467820.426),0,...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709444.553 8466709.353),135357.990228,1.343910,1.217812,1.125507,5559104498,POINT (-75.15400 39.96258),POINT (-5708792.439 8467804.519),25.108
4,31960,1154.164354,835.304338,930.362886,1014.141346,3838,6,POINT (-75.15417 39.96250),POINT (-5708811.866 8467820.426),0,...,"MULTIPOLYGON (((-75.14967 39.94550, -75.14974 ...",POINT (-5709506.409 8466898.631),143083.338564,1.381729,1.240553,1.138071,5559104498,POINT (-75.15400 39.96258),POINT (-5708792.439 8467804.519),25.108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187898,2477389,924.965866,800.614592,840.086946,873.061474,2653,5528,POINT (-75.08417 40.01500),POINT (-5697664.520 8461972.438),442,...,"POLYGON ((-75.07740 40.01298, -75.07381 40.013...",POINT (-5697723.785 8461049.372),100033.999718,1.155320,1.101036,1.059451,5549509413,POINT (-75.08425 40.01494),POINT (-5697677.606 8461979.672),14.953
187899,2482076,942.629242,810.461704,852.318604,887.355293,2653,5529,POINT (-75.08417 40.01500),POINT (-5697664.520 8461972.438),442,...,"POLYGON ((-75.07740 40.01298, -75.07381 40.013...",POINT (-5697724.026 8461031.689),101381.810946,1.163077,1.105959,1.062291,5549509413,POINT (-75.08425 40.01494),POINT (-5697677.606 8461979.672),14.953
187900,2486763,930.335289,801.632228,842.422474,876.544087,2653,5530,POINT (-75.08417 40.01500),POINT (-5697664.520 8461972.438),442,...,"POLYGON ((-75.07740 40.01298, -75.07381 40.013...",POINT (-5697720.500 8461043.788),100941.993042,1.160551,1.104357,1.061367,5549509413,POINT (-75.08425 40.01494),POINT (-5697677.606 8461979.672),14.953
187901,2491450,940.228763,815.698480,855.258921,888.284289,2653,5531,POINT (-75.08417 40.01500),POINT (-5697664.520 8461972.438),442,...,"POLYGON ((-75.07740 40.01298, -75.07381 40.013...",POINT (-5697736.682 8461034.982),99575.153417,1.152667,1.099350,1.058477,5549509413,POINT (-75.08425 40.01494),POINT (-5697677.606 8461979.672),14.953


In [10]:
# Number of combinations within range per city
for i in range(len(RoadComb)): print(cities[i], len(RoadComb[i]))

Philadelphia, United States 187903
Denver, United States 191169
Ghent, Belgium 278997
Amsterdam, Netherlands 315906
Dhaka Metropolitan, Bangladesh 5471
Dublin, Ireland 442462
Vancouver, Canada 473119
Tel Aviv, Israel 237205
Washington DC, United States 247715


In [ ]:
# Block 7 calculate route networks of all grid-parkentry combinations within euclidean threshold distance

warnings.filterwarnings("ignore")
start_time = time.time()

Routes = list()
for j in range(len(cities)):
    Graph = graphs[j]
    CityRoads = RoadComb[j] # iloc to test the iteration speed.
    PR = road_nodes[j]
    
    block = 250000

    Route_parts = pd.DataFrame()
    len2 = int(np.ceil(len(CityRoads)/block))
    
    # Divide in chunks of 250000 for computational load
    for k in range(len2):    
        CityRoad = CityRoads.iloc[k*block:k*block+block]

        parknode = list(CityRoad['Parkroad_osmid'])
        gridnode = list(CityRoad['grid_osm'])
        
        s_mat = list([])
        s_mat1 = list([])
        s_mat2 = list([])
        s_mat3 = list([])
        s_mat4 = list([])
        len1 = len(CityRoad)
        
        print(cities[j].rsplit(',')[0], k+1,'/',len2,'range',k*block,'-',k*block+np.where(k*block+block >= len1,len1,block))
        for i in range(len(CityRoad)):
            try:
                shortest = nx.shortest_path(Graph, gridnode[i], parknode[i], 'travel_dist', method = 'dijkstra')
                s_mat.append(shortest)
                shortest_to = list(shortest[1:len(shortest)])
                shortest_to.append(0)
                s_mat1.append(shortest_to)
                s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                s_mat3.append(list(np.arange(0, len(shortest))))
                s_mat4.append('normal way')
            except:
                try:
                    # Check the reverse
                    shortest = nx.shortest_path(Graph, parknode[i], gridnode[i], 'travel_dist', method = 'dijkstra')
                    s_mat.append(shortest)
                    shortest_to = list(shortest[1:len(shortest)])
                    shortest_to.append(0)
                    s_mat1.append(shortest_to)
                    s_mat2.append(list(np.repeat(i+block*k, len(shortest))))
                    s_mat3.append(list(np.arange(0, len(shortest))))
                    s_mat4.append('reverse way')
                except:
                    # Otherwise the nearest node is taken, which is iterated 10 times at max. Order in route for nearest node:
                    # 1. gridnode to nearest to the original failed parknode
                    # 2. The reverse of 1.
                    # 3. nearest gridnode to the failed one and route to park
                    # 4. The reverse of 3.

                    len3 = 0
                    alt_route = list([])
                    while len3 < 25 and len(alt_route) < 1:

                        len3 = len3 +1
                        # Grid nearest
                        g_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['grid_osm'])]['geometry']
                        g_nearest = pd.DataFrame((abs(float(g_geom.x) - PR['geometry'].x)**2
                        +abs(float(g_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        g_grid = g_nearest.iloc[len3,1]
                        g_park = CityRoad.iloc[i]['Parkroad_osmid']

                        p_geom = PR[PR['osmid_var'] == int(CityRoad.iloc[i:i+1]['Parkroad_osmid'])]['geometry']
                        p_nearest = pd.DataFrame((abs(float(p_geom.x) - PR['geometry'].x)**2
                        +abs(float(p_geom.y) - PR['geometry'].y)**2)**(1/2)
                                                ).join(PR['osmid_var']).sort_values(0)

                        p_grid = CityRoad.iloc[i]['grid_osm']
                        p_park = p_nearest.iloc[len3,1]

                        try:
                            alt_route.append(nx.shortest_path(Graph, p_grid, p_park, 
                                                              'travel_dist', method = 'dijkstra'))
                        except:
                            try:
                                alt_route.append(nx.shortest_path(Graph, p_park, p_grid, 
                                                                  'travel_dist', method = 'dijkstra'))
                            except:
                                try:
                                    alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                      'travel_dist', method = 'dijkstra'))
                                except:
                                    try:
                                        alt_route.append(nx.shortest_path(Graph, g_grid, g_park, 
                                                                          'travel_dist', method = 'dijkstra'))
                                    except:
                                        if len3 == 10:
                                            print(i+block*k,'No route between grid and park-entry and both their 10 alternatives')
                                            pass
                                        pass
                    len4 = len(alt_route)
                    if len4 > 0: 
                        print('for index',i,'nearest node found between', 
                                               alt_route[0][0],'and',alt_route[0][-1])
                        
                        s_mat.append(alt_route[0])
                        shortest_to = list(alt_route[0][1:len(alt_route[0])])
                        shortest_to.append(0)
                        s_mat1.append(shortest_to)
                        s_mat2.append(list(np.repeat(i+block*k,len4)))
                        s_mat3.append(list(np.arange(0, len4)))
                        s_mat4.append('altered way')
                    else:
                        s_mat.append(-1)
                        s_mat1.append(-1)
                        s_mat2.append(i+block*k)
                        s_mat3.append(-1)
                        s_mat4.append('no way')

            if i % 10000 == 0: print(round((i+block*k)/len(CityRoads)*100,2),'% done',
                                     round((time.time() - start_time) / 60,2),'mns')

        print(round((i+block*k)/len(CityRoads)*100,2),'% pathfinding done', round((time.time() - start_time) / 60,2),' mns')
                          
        # Unpack lists
        s_mat_u = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat) for i in b]
        s_mat_u1 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat1) for i in b]
        s_mat_u2 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat2) for i in b]
        s_mat_u3 = [i for b in map(lambda x:[x] if not isinstance(x, list) else x, s_mat3) for i in b]

        # Format df
        routes = pd.DataFrame([s_mat_u,s_mat_u1,s_mat_u2,s_mat_u3]).transpose()
        routes.columns = ['from','to','route','step']
        mat_key = list([])
        for n in range(len(routes)):
            mat_key.append(str(int(s_mat_u[n])) + '-' + str(int(s_mat_u1[n])))
        routes['key'] = mat_key
        routes = routes.set_index('key')

        # Add route information
        routes = routes.join(road_conn[j], how = 'left')
        routes = gpd.GeoDataFrame(routes, geometry = 'geometry', crs = 4326)
        print('formatting done', round((time.time() - start_time) / 60,2), 'mns')
        routes = routes.sort_values(by = ['route','step'])

        # get single (dissolved) line per route, attach information.
        routes2 = routes[['route','geometry']].dissolve('route')
        routes2['way_calculated'] = s_mat4
        routes2['route_cost'] = routes.groupby('route')['length'].sum()
        routes2['num_steps'] = routes.groupby('route')['step'].max()
        routes2['index'] = CityRoad.index
        routes2 = routes2.set_index(['index'])
        routes2.index = routes2.index.astype(int)
        routes2 = pd.merge(routes2, CityRoad[['Grid_No','Park_No','Park_entry_No','grid_entry_dist','Parkroad_osmid',
                                              'grid_osm','walk_area_m2','size_infl_proot2','size_infl_proot3',
                                              'size_infl_proot5','raw euclidean']],
                                                
                            left_index = True, right_index = True)
        routes2['raw_total_cost'] = routes2['route_cost'] + routes2['grid_entry_dist']
        routes2['grav2_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot2']
        routes2['grav3_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot3']
        routes2['grav5_total_cost'] = (routes2['route_cost'] + routes2['grid_entry_dist']) / routes2['size_infl_proot5']

        routes2['gridpark_no'] = routes2['Grid_No'].astype(str) +'-'+ routes2['Park_No'].astype(str)
        print('dissolving done', round((time.time() - start_time) / 60,2), 'mns')
        Route_parts = pd.concat([Route_parts, routes2])
    Routes.append(Route_parts)
Routes[0]

Philadelphia 1 / 1 range 0 - 187903
0.0 % done 0.0 mns
5.32 % done 0.31 mns
10.64 % done 0.66 mns
15.97 % done 0.89 mns
21.29 % done 1.12 mns
26.61 % done 1.39 mns
31.93 % done 1.77 mns
37.25 % done 1.91 mns
42.58 % done 2.13 mns
47.9 % done 2.33 mns
53.22 % done 2.52 mns
58.54 % done 2.73 mns
63.86 % done 3.03 mns
69.18 % done 3.27 mns
74.51 % done 3.44 mns
79.83 % done 3.54 mns
85.15 % done 3.7 mns
90.47 % done 3.87 mns
95.79 % done 4.04 mns
100.0 % pathfinding done 4.17  mns
formatting done 5.46 mns
dissolving done 6.2 mns
Denver 1 / 1 range 0 - 191169
0.0 % done 6.2 mns
5.23 % done 6.64 mns
10.46 % done 6.99 mns
15.69 % done 7.58 mns
20.92 % done 8.16 mns
26.15 % done 8.77 mns
31.39 % done 9.44 mns
36.62 % done 10.14 mns
41.85 % done 10.63 mns
47.08 % done 11.05 mns
52.31 % done 11.21 mns
57.54 % done 11.34 mns
62.77 % done 11.45 mns
68.0 % done 11.59 mns
73.23 % done 11.86 mns
78.46 % done 12.21 mns
83.7 % done 12.41 mns
88.93 % done 12.94 mns
94.16 % done 13.25 mns
99.39 % done 1

In [ ]:
# Block 8 determine best parkentry points from each grid, then calculate grid scores
# and finally aggregate city access in categories (high, medium, low and no access)

start_time = time.time()
popg_acc = pd.DataFrame()
adj_popg_acc = pd.DataFrame()
grid_scores = list([])
gridpark = list([])
for n in range(len(cities)):    
    print(cities[n])
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    grid_score = list([])
    gridparks = list([])
    gridpark.append(gridparks)
    popgrid_access = pd.DataFrame()
    adj_popgrid_access = pd.DataFrame()
    for i in range(len(l1)):
        # Get the lowest indices grouped by a key consisting of grid no and park no (best entry point from a grid to a park)
        str1 = 'gridpark_' + l1[i]
        locals()[str1] = Routes[n].iloc[Routes[n].groupby('gridpark_no')[(str(l1[i]) +'_total_cost')].idxmin()]
        l2 = list()
        
        # Get the total cost column
        for j in enumerate(l1): 
            if j[0] != i: l2.append(j[1] + '_total_cost')
        locals()[str1] = locals()[str1].loc[:, ~locals()[str1].columns.isin(l2)]
        
        # Get grid information
        locals()[str1] = pd.merge(locals()[str1], grids[n][['PoP2015_Number','geometry']],
                                left_on = 'Grid_No', right_index = True, how = 'outer')
        locals()[str1] = locals()[str1].reset_index()
        
        # formatting
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].fillna(-1)
        locals()[str1]['Park_No'] = locals()[str1]['Park_No'].astype(int)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].fillna(-1)
        locals()[str1]['Park_entry_No'] = locals()[str1]['Park_entry_No'].astype(int)
        
        grdsc = pd.DataFrame()
        gridsc = pd.DataFrame()
        print(m1[i], round((time.time() - start_time) / 60,2), 'mns')
        
        # For each threshold given, calculate a score
        for k in range(len(thresholds)):
            t = thresholds[k]
            str2 = str(t)
            score = 'tr_'+ str2
            adj_score = 'adj_tr_' + str2
            
            #Only get routes within the threshold given (it loops over every threshold) and calculate the scores
            thold = locals()[str1][locals()[str1][l1[i] + '_total_cost'] <= t]
            thold[score] = t - thold[l1[i] + '_total_cost']
            thold['pop' + score] = thold[score] * thold['PoP2015_Number']
            thold[adj_score] = thold[score] * ((math.pi*600**2) / (math.pi*t**2)) / (600/t)
            thold['walk_area_ha' + str2] = locals()[str1]['walk_area_m2'] /10000
            thold['walkha_person' + str2] = thold['PoP2015_Number'] / thold['walk_area_ha' + str2]
            #thold = thold[thold[score] > 0]
            
            # Join the gridpark information from before.
            locals()[str1] = locals()[str1].join(thold[[score,'pop' + score,adj_score,'walk_area_ha' + str2, 'walkha_person' + str2]])
            # get the grid_scores
            gs = pd.DataFrame()
            gs[[score,'pop_' + score,adj_score,'walkha_' + score,'walkha_person_' + score]] = locals()[str1].groupby(
                    'Grid_No')[score,'pop' + score, adj_score, 'walk_area_ha' + str2, 'walkha_person' + str2].sum()
            
            trstr = locals()[str1][locals()[str1][score] > 0]
            gs[score + '_parks'] = trstr.groupby('Grid_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            gs[score + '_routes'] = gpd.GeoDataFrame(trstr[['Grid_No','geometry_x']],
                                                          geometry = 'geometry_x', crs = 4326).dissolve('Grid_No')

            # Add parks which grids have access to with its closest access point
            gs[score+'Park:entry'] = trstr[trstr['Park_No'] >=0].groupby('Grid_No')['Park_No'].apply(list).astype(str
            ) + ':' + trstr[trstr['Park_entry_No'] >=0].groupby('Grid_No')['Park_entry_No'].apply(list).astype(str)
            
            # determine the thresholds category-score. 
            # High >= threshold (perfect score to one park), medium is above half perfect, 
            # low is below this and no is no access to a park for a certain grid within the threshold given
            gs[score+'_access'] = np.select([gs[score] >= t, (gs[score] < t) & (
            gs[score]>= t/2), (gs[score] < t/2) & (gs[score]> 0), gs[score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
                        
            gs[score+'_adj_access'] = np.select([gs[adj_score] >= t, (gs[adj_score] < t) & (
            gs[adj_score]>= t/2), (gs[adj_score] < t/2) & (gs[adj_score]> 0), gs[adj_score] <= 0],
                  ['1 high','2 medium','3 low','4 no'])
            
            gs = gs.join(grids[n]['PoP2015_Number'], how = 'outer')
            grdsc = pd.concat([grdsc, gs], axis = 1)
            
            gs = gpd.GeoDataFrame(gs, geometry = score + '_routes', crs = 4326)
            gs.to_file('D:Dumps/Scores output WP-OSM/Grid_lines_shp/gridscore_'+ l1[i] + '_' + str2 + '_' + cities[n] + '.shp')
            
            gsc = gs.loc[:,~gs.columns.isin([score + '_routes'])]
            gridsc = pd.concat([gridsc, gsc])

            # Group according to the categories just created and sum the populations living in those grids
            popgacc = pd.DataFrame()
            popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_access')['PoP2015_Number'].sum()
            popgrid_access = pd.concat([popgrid_access, popgacc],axis=1)   
            
            adj_popgacc = pd.DataFrame()
            adj_popgacc[m1[i]+'_'+str(t)] = gs.groupby(score+'_adj_access')['PoP2015_Number'].sum()
            adj_popgrid_access = pd.concat([adj_popgrid_access, adj_popgacc],axis=1)   
            print('grid ',t)
        
        grid_score.append(grdsc)
        
        gridsc = gridsc.join(grids[n]['geometry'])
        gridsc = gpd.GeoDataFrame(gridsc, geometry = 'geometry', crs = 4326)
        gridsc.to_file('D:Dumps/Scores output WP-OSM/Grid_geoms_shp/gridscore_'+ l1[i] + '_' + cities[n] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        gridsc = gridsc.loc[:, gridsc.columns!='geometry']
        gridsc.to_csv('D:/Dumps/Scores output WP-OSM/Grid_csv/gridscore_'+ l1[i] + '_' + cities[n] + '.csv')
        gridparks.append(locals()[str1])
        
    grid_scores.append(grid_score)

    # For each city, divide the population access by group by the total to get its share.
    popgrid_access = popgrid_access / popgrid_access.sum()
    popgrid_access = pd.DataFrame(popgrid_access.unstack())
    popg_acc = pd.concat([popg_acc, popgrid_access], axis = 1)
    
    adj_popgrid_access = adj_popgrid_access / adj_popgrid_access.sum()
    adj_popgrid_access = pd.DataFrame(adj_popgrid_access.unstack())
    adj_popg_acc = pd.concat([adj_popg_acc, adj_popgrid_access], axis = 1)
    
    print(cities[n],'done', round((time.time() - start_time) / 60,2), 'mns')
popg_acc.columns = cities
adj_popg_acc.columns = cities

popg_acc.to_csv('D:/Dumps/Scores output WP-OSM/popgrid_access.csv')
adj_popg_acc.to_csv('D:/Dumps/Scores output WP-OSM/radius-euclidean adjusted popgrid access.csv')

adj_popg_acc    


In [ ]:
# Block 9 calculte park scores from previously determined best grid-park routes.

start_time = time.time()
cityparks = list([])
for i in range(len(cities)):
    
    # For the four distance decay variants regarding park size.
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    parks = list([])
    for j in range(len(l1)):
        parksc = pd.DataFrame()
        prksc = pd.DataFrame()
        for k in range(len(thresholds)):
            score = 'tr_' + str(thresholds[k])
            str2 = str(thresholds[k])
            str1 = gridpark[i][j][gridpark[i][j][score] > 0]

            # Get the park scores
            prk = pd.DataFrame()
            prk[[score,'pop_' + score,'walkha_' + score]] = str1.groupby(
                'Park_No')[score,'pop' + score,'walk_area_ha' + str2].sum()
            prk[score + '_parks'] = str1.groupby('Park_No')['gridpark_no'].count()
            
            # Add the routes as a dissolved line_geom
            prk[score+'route'] = gpd.GeoDataFrame(str1[['Park_No','geometry_x']], 
                             geometry = 'geometry_x', crs = 4326).dissolve('Park_No')
            
            # Add parks which grids have access to with its closest access point
            prk[score+'Grid:Pentry'] = str1[str1['Grid_No'] >=0].groupby('Park_No')['Grid_No'].apply(list).astype(str
            ) + ':' + str1[str1['Park_entry_No'] >=0].groupby('Park_No')['Park_entry_No'].apply(list).astype(str)
            
            # Get all parks, even with no score.
            prk = prk.join(parks_in_range[i].iloc[:,0], how = 'outer')
            prk = prk.loc[:,~prk.columns.isin(['components'])]
            #prk = prk.fillna(-1)
            print('park', thresholds[k])
            
            # Get the park score categories (same as grid score)
            prk[score+'_access'] = np.select([prk[score] >= t, (prk[score] < t) & (
                prk[score]>= t/2), (prk[score] < t/2) & (prk[score]> 0), prk[score] <= 0 | prk[score].isna()],
                ['1 high','2 medium','3 low','4 no'])

            parksc = pd.concat([parksc, prk], axis = 1)
                
            prk = gpd.GeoDataFrame(prk, geometry = score+'route', crs = 4326)
            prk.to_file('D:Dumps/Scores output WP-OSM/Park_lines_shp/parkscore_'+ l1[j] + '_' + str2 + '_' + cities[i] + '.shp')
            
            psc = prk.loc[:,~prk.columns.isin([score + 'route'])]
            prksc = pd.concat([prksc, psc])
            
        parks.append(parksc)
        
        prksc = prksc.join(parks_in_range[i]['geometry'])
        prksc = gpd.GeoDataFrame(prksc, geometry = 'geometry', crs = 4326)
        prksc.to_file('D:Dumps/Scores output WP-OSM/Grid_geoms_shp/parkscore_'+ l1[j] + '_' + cities[i] + '.shp')
        
        # Detailed scores to files number of cities * ways to measure = number of files.
        # Different threshold-scores are in the same dataframe
        prksc = prksc.loc[:, prksc.columns!='geometry']
        prksc.to_csv('D:/Dumps/Scores output WP-OSM/Park_csv/parkscore_'+ l1[j] + '_' + cities[i]+ '.csv')

        print(m1[j], round((time.time() - start_time) / 60,2), 'mns')
    cityparks.append(parks)
    print(cities[i],'done', round((time.time() - start_time) / 60,2), 'mns')
pd.DataFrame(cityparks[1][1])

In [ ]:
# Block 10 get the preferenced parks for each grid (lowest score) for all distance decay variants.
preference = list([])
for n in enumerate(cities): 
    print(n[1])
    l1 = list(['raw','grav2','grav3','grav5'])
    m1 = list(['entrance','gravity**(1/2)','gravity**(1/3)','gravity**(1/5)'])
    prefer = list([])
    for j in enumerate(l1):
        pref = list([])
        print(m1[j[0]])
        for k in thresholds:
            score = 'tr_'+ str(k)
            g = gridpark[n[0]][j[0]].iloc[gridpark[n[0]][j[0]].groupby('Grid_No')[score].idxmax().dropna().astype(int)]
            g = g[g[score] > 0]
            g.join(grids[n[0]]['grid_id'], how = 'outer')
            g.loc[:, (g.columns!='geometry_x') & (g.columns!='geometry_y')].to_csv('D:/Dumps/Scores output WP-OSM/Grid_pref_parks_csv/park-pref_' + j[1] +'-'+ str(k) + '-' + n[1] +'.csv')
            g_lines = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_y'])], geometry = 'geometry_x', crs = 4326)
            g_lines.to_file('D:Dumps/Scores output WP-OSM/Grid_pref_parks_lines/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            g_geoms = gpd.GeoDataFrame(g.loc[:, ~g.columns.isin(['geometry_x'])], geometry = 'geometry_y', crs = 4326)
            g_geoms.to_file('D:Dumps/Scores output WP-OSM/Grid_pref_parks_geoms/park-pref_' +'-'+ j[1] + str(k) + '-' + n[1] +'.shp')
            pref.append(g)
            print('park_prefer',k)
        prefer.append(pref)
        len(pref)
    preference.append(prefer)
print('all done')